In [ ]:
import uproot
import awkward as ak
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema, DelphesSchema
from coffea import hist, processor

In [ ]:
from tools.helpers import *

In [ ]:
events_150 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb_1500_XX_10_v4/2HDMa_bb_1500_150_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_250 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb_1500_XX_10_v4/2HDMa_bb_1500_250_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_350 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb_1500_XX_10_v4/2HDMa_bb_1500_350_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_500 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb_1500_XX_10_v4/2HDMa_bb_1500_500_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()   

events_750 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb_1500_XX_10_v4/2HDMa_bb_1500_750_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
electron = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_750.elec_pt,
            eta = events_750.elec_eta,
            phi = events_750.elec_phi,
            M = events_750.elec_mass,
            copy = False,
        )
electron['id'] = events_750.elec_idpass  # > 0 should be loose
electron['iso'] = events_750.elec_isopass
electron['charge'] = events_750.elec_charge

ele_l = electron[((electron['id']>0)&(electron['iso']>0)&(electron.pt>10)&(np.abs(electron.eta)<3))]
        
        #muons
        
muon = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_750.muon_pt,
            eta = events_750.muon_eta,
            phi = events_750.muon_phi,
            M = events_750.muon_mass,
            copy = False,
        )
muon['id'] = events_750.muon_idpass  # > 0 should be loose
muon['iso'] = events_750.muon_isopass
muon['charge'] = events_750.muon_charge

muon_l = muon[((muon['id']>0)&(muon['iso']>0)&(muon.pt>4)&(np.abs(muon.eta)<2.8))]
        
#taus
        
tau = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_750.tau_pt,
            eta = events_750.tau_eta,
            phi = events_750.tau_phi,
            M = events_750.tau_mass,
            copy = False,
        )
tau['iso'] = events_750.tau_isopass   # > 0 should be loose
tau['charge'] = events_750.tau_charge

tau_l = tau[((tau['iso']>0)&(tau.pt>30)&(np.abs(tau.eta)<3))]
        
#photons
        
gamma = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_750.gamma_pt,
            eta = events_750.gamma_eta,
            phi = events_750.gamma_phi,
            M = events_750.gamma_mass,
            copy = False,
        )
gamma['id'] = events_750.gamma_idpass  # > 0 should be loose
gamma['iso'] = events_750.gamma_isopass

gamma_l = gamma[((gamma['id']>0)&(gamma['iso']>0)&(gamma.pt>20)&(np.abs(gamma.eta)<3))]


fatjet = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_750.fatjet_pt,
            eta = events_750.fatjet_eta,
            phi = events_750.fatjet_phi,
            M = events_750.fatjet_mass,
            copy = False,
        )

met = ak.flatten(events_750.metpuppi_pt)

In [ ]:
lead_fatjet = fatjet[:,0:1]

In [ ]:
gen = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_750.genpart_pt,
            eta = events_750.genpart_eta,
            phi = events_750.genpart_phi,
            M = events_750.genpart_mass,
            copy = False,
        )
gen['pdgId'] = events_750.genpart_pid
#gen['status'] = events.genpart_status
higgs = gen[(gen.pdgId==25)][:,-1:]  # only keep the last copy. status codes seem messed up?

matched_jet = fatjet[match(fatjet, higgs, deltaRCut=0.8)]

In [ ]:
delta_r_paf(matched_jet[:,0:1], lead_fatjet)

In [ ]:
ak.to_numpy(delta_r_paf(matched_jet[:,0:1], lead_fatjet))

In [ ]:
plt.hist(ak.to_numpy(delta_r_paf(matched_jet[:,0:1], lead_fatjet)), np.linspace(0,1,10), histtype = 'step', label = '1500_750_10', log=True)

In [ ]:
plt.hist(delta_r_paf(matched_jet[:,0:1], lead_fatjet), np.linspace(0,1,11), histtype = 'step', label = '1500_750_10', log=True)

In [ ]:
met_sel = (met>200)
ele_sel = ((ak.num(ele_l)==0) & met_sel)
mu_sel = ((ak.num(muon_l)==0) & ele_sel)
tau_sel = ((ak.num(tau_l)==0) & mu_sel)
baseline = ((ak.num(gamma_l)==0) & tau_sel)

In [ ]:
fatjet[ak.argsort(fatjet.pt, ascending=False)][:,0:1].pt

In [ ]:
fatjet.pt[baseline]

In [ ]:
len(met[baseline])

In [ ]:
electron_150_pt = events_150.elec_pt[(events_150.elec_pt>10) & (abs(events_150.elec_eta)<3) & (events_150.elec_isopass>0) & (events_150.elec_idpass>0)]
electron_250_pt = events_250.elec_pt[(events_250.elec_pt>10) & (abs(events_250.elec_eta)<3) & (events_250.elec_isopass>0) & (events_250.elec_idpass>0)]
electron_350_pt = events_350.elec_pt[(events_350.elec_pt>10) & (abs(events_350.elec_eta)<3) & (events_350.elec_isopass>0) & (events_350.elec_idpass>0)]
electron_500_pt = events_500.elec_pt[(events_500.elec_pt>10) & (abs(events_500.elec_eta)<3) & (events_500.elec_isopass>0) & (events_500.elec_idpass>0)]
electron_750_pt = events_750.elec_pt[(events_750.elec_pt>10) & (abs(events_750.elec_eta)<3) & (events_750.elec_isopass>0) & (events_750.elec_idpass>0)]

In [ ]:
muon_150_pt = events_150.muon_pt[(events_150.muon_pt>4) &(abs(events_150.muon_eta)<2.8) & (events_150.muon_isopass>0) & (events_150.muon_idpass>0)]
muon_250_pt = events_250.muon_pt[(events_250.muon_pt>4) &(abs(events_250.muon_eta)<2.8) & (events_250.muon_isopass>0) & (events_250.muon_idpass>0)]
muon_350_pt = events_350.muon_pt[(events_350.muon_pt>4) &(abs(events_350.muon_eta)<2.8) & (events_350.muon_isopass>0) & (events_350.muon_idpass>0)]
muon_500_pt = events_500.muon_pt[(events_500.muon_pt>4) &(abs(events_500.muon_eta)<2.8) & (events_500.muon_isopass>0) & (events_500.muon_idpass>0)]
muon_750_pt = events_750.muon_pt[(events_750.muon_pt>4) &(abs(events_750.muon_eta)<2.8) & (events_750.muon_isopass>0) & (events_750.muon_idpass>0)]

In [ ]:
tau_150_pt = events_150.tau_pt[(events_150.tau_pt>30) &(abs(events_150.tau_eta)<3) & (events_150.tau_isopass>0)]
tau_250_pt = events_250.tau_pt[(events_250.tau_pt>30) &(abs(events_250.tau_eta)<3) & (events_250.tau_isopass>0)]
tau_350_pt = events_350.tau_pt[(events_350.tau_pt>30) &(abs(events_350.tau_eta)<3) & (events_350.tau_isopass>0)]
tau_500_pt = events_500.tau_pt[(events_500.tau_pt>30) &(abs(events_500.tau_eta)<3) & (events_500.tau_isopass>0)]
tau_750_pt = events_750.tau_pt[(events_750.tau_pt>30) &(abs(events_750.tau_eta)<3) &(events_750.tau_isopass>0)]

In [ ]:
gamma_150_pt = events_150.gamma_pt[(events_150.gamma_pt>20) &(abs(events_150.gamma_eta)<3) & (events_150.gamma_isopass>0) & (events_150.gamma_idpass>0)]
gamma_250_pt = events_250.gamma_pt[(events_250.gamma_pt>20) &(abs(events_250.gamma_eta)<3) & (events_250.gamma_isopass>0) & (events_250.gamma_idpass>0)]
gamma_350_pt = events_350.gamma_pt[(events_350.gamma_pt>20) &(abs(events_350.gamma_eta)<3) & (events_350.gamma_isopass>0) & (events_350.gamma_idpass>0)]
gamma_500_pt = events_500.gamma_pt[(events_500.gamma_pt>20) &(abs(events_500.gamma_eta)<3) & (events_500.gamma_isopass>0) & (events_500.gamma_idpass>0)]
gamma_750_pt = events_750.gamma_pt[(events_750.gamma_pt>20) &(abs(events_750.gamma_eta)<3) & (events_750.gamma_isopass>0) & (events_750.gamma_idpass>0)]

In [ ]:
plt.hist(ak.num(electron_150_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_150_10', log=True)
plt.hist(ak.num(electron_250_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_250_10', log=True)
plt.hist(ak.num(electron_350_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_350_10', log=True)
plt.hist(ak.num(electron_500_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_500_10', log=True)
plt.hist(ak.num(electron_750_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_750_10', log=True)
plt.xlabel(r'$N_{electron}$')
plt.ylabel(r'Events')
plt.legend()

In [ ]:
plt.hist(ak.num(electron_150_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_150_10')
plt.hist(ak.num(electron_250_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_250_10')
plt.hist(ak.num(electron_350_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_350_10')
plt.hist(ak.num(electron_500_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_500_10')
plt.hist(ak.num(electron_750_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_750_10')
plt.xlabel(r'$N_{electron}$')
plt.ylabel(r'Events')
plt.legend()

In [ ]:
plt.hist(ak.num(muon_150_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_150_10', log=True)
plt.hist(ak.num(muon_250_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_250_10', log=True)
plt.hist(ak.num(muon_350_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_350_10', log=True)
plt.hist(ak.num(muon_500_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_500_10', log=True)
plt.hist(ak.num(muon_750_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_750_10', log=True)
plt.xlabel(r'$N_{muon}$')
plt.ylabel(r'Events')
plt.legend()

In [ ]:
plt.hist(ak.num(muon_150_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_150_10')
plt.hist(ak.num(muon_250_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_250_10')
plt.hist(ak.num(muon_350_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_350_10')
plt.hist(ak.num(muon_500_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_500_10')
plt.hist(ak.num(muon_750_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_750_10')
plt.xlabel(r'$N_{electron}$')
plt.ylabel(r'Events')
plt.legend()

In [ ]:
plt.hist(ak.num(tau_150_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_150_10', log=True)
plt.hist(ak.num(tau_250_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_250_10', log=True)
plt.hist(ak.num(tau_350_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_350_10', log=True)
plt.hist(ak.num(tau_500_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_500_10', log=True)
plt.hist(ak.num(tau_750_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_750_10', log=True)
plt.xlabel(r'$N_{\tau}$')
plt.ylabel(r'Events')
plt.legend()

In [ ]:
plt.hist(ak.num(tau_150_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_150_10')
plt.hist(ak.num(tau_250_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_250_10')
plt.hist(ak.num(tau_350_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_350_10')
plt.hist(ak.num(tau_500_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_500_10')
plt.hist(ak.num(tau_750_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_750_10')
plt.xlabel(r'$N_{\tau}$')
plt.ylabel(r'Events')
plt.legend()

In [ ]:
plt.hist(ak.num(gamma_150_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_150_10', log=True)
plt.hist(ak.num(gamma_250_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_250_10', log=True)
plt.hist(ak.num(gamma_350_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_350_10', log=True)
plt.hist(ak.num(gamma_500_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_500_10', log=True)
plt.hist(ak.num(gamma_750_pt), np.linspace(-0.5,5.5,7), histtype = 'step', label = '1500_750_10', log=True)
plt.xlabel(r'$N_{\gamma}$')
plt.ylabel(r'Events')
plt.legend()

In [ ]:
puppiMET = [[],[],[],[], np.linspace(0,250,11), r'$Puppi\ MET_{pt}$']
genMET = [[],[],[],[], np.linspace(0,80,9), r'$Gen\ MET_{pt}$']
fatjet_pt = [[],[],[],[], np.linspace(200,450,26), r'$Fatjet\ p_{T}$']
fatjet_eta = [[],[],[],[], np.linspace(-4,4,9), r'$Fatjet\ \eta$']
fatjet_phi = [[],[],[],[], np.linspace(-4,4,9), r'$Fatjet\ \phi$']
fatjet_mass = [[],[],[],[], np.linspace(0,200,21), r'$Fatjet\ mass$']
fatjet_msoftdrop = [[],[],[],[], np.linspace(0,150,16), r'$Fatjet\ sd\ mass$']
fatjet_tau1 = [[],[],[],[], np.linspace(0,0.6,13), r'$Fatjet\ \tau_1$']
fatjet_tau2 = [[],[],[],[], np.linspace(0,0.25,6), r'$Fatjet\ \tau_2$']
fatjet_tau3 = [[],[],[],[], np.linspace(0,0.2,5), r'$Fatjet\ \tau_3$']
fatjet_tau4 = [[],[],[],[], np.linspace(0,0.15,4), r'$Fatjet\ \tau_4$']
varis = {'metpuppi_pt': puppiMET, 'genmet_pt': genMET, 'fatjet_pt': fatjet_pt, 'fatjet_eta': fatjet_eta, 
         'fatjet_phi': fatjet_phi, 'fatjet_mass': fatjet_mass, 'fatjet_msoftdrop': fatjet_msoftdrop,
        'fatjet_tau1': fatjet_tau1, 'fatjet_tau2': fatjet_tau2, 'fatjet_tau3': fatjet_tau3, 'fatjet_tau4': fatjet_tau4}

In [ ]:
plt.hist(ak.flatten(events_150_4.fatjet_pt), np.linspace(200,600,40), histtype = 'step', label = '1500_150_10')
plt.hist(ak.flatten(events_250_4.fatjet_pt), np.linspace(200,600,40), histtype = 'step', label = '1500_250_10')
plt.hist(ak.flatten(events_350_4.fatjet_pt), np.linspace(200,600,40), histtype = 'step', label = '1500_350_10')
plt.hist(ak.flatten(events_500_4.fatjet_pt), np.linspace(200,600,40), histtype = 'step', label = '1500_500_10')
plt.hist(ak.flatten(events_750_4.fatjet_pt), np.linspace(200,600,40), histtype = 'step', label = '1500_750_10')
plt.xlabel(r'$fatjet\ P_{T}$')
plt.ylabel(r'Events')
plt.legend()

In [ ]:
fatjet_events = events_42[ak.num(events_42.fatjet_pt)>0]
print(fatjet_events.fatjet_pt)

In [ ]:
ak.flatten(fatjet_events.fatjet_msoftdrop[ak.argsort(fatjet_events.fatjet_pt, ascending=False)][:,0:1], axis=1)

In [ ]:
varis

In [ ]:
for vari in varis:
    varis[vari][0] = ak.flatten(ak.concatenate([events_11[vari],events_12[vari]]))
    varis[vari][1] = ak.flatten(ak.concatenate([events_21[vari],events_22[vari]]))
    varis[vari][2] = ak.flatten(ak.concatenate([events_31[vari],events_32[vari]]))
    varis[vari][3] = ak.flatten(ak.concatenate([events_41[vari],events_42[vari]]))

In [ ]:
varis

In [ ]:
len(varis['metpuppi_pt'][3])

In [ ]:
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

In [ ]:
for vari in varis:
    plt.hist(varis[vari][0], bins=varis[vari][4], histtype = 'step', label = 'MH3_1500_MH4_150_Mchi_10')
    plt.hist(varis[vari][1], bins=varis[vari][4], histtype = 'step', label = 'MH3_1500_MH4_250_Mchi_10')
    plt.hist(varis[vari][2], bins=varis[vari][4], histtype = 'step', label = 'MH3_1500_MH4_300_Mchi_10') 
    plt.hist(varis[vari][3], bins=varis[vari][4], histtype = 'step', label = 'MH3_1500_MH4_400_Mchi_10')
    plt.xlabel(varis[vari][5])
    plt.ylabel(r'Events')
    plt.legend(prop={'size': 13})
    plt.savefig('/home/users/ewallace/public_html/HbbMET/delphes_'+str(vari)+'.png')
    plt.clf()

In [ ]:
from analysis.test import meta_processor

In [ ]:
from yaml import Loader, Dumper
import yaml

In [ ]:
with open('../data/samples.yaml', 'r') as f:
    samples = yaml.load(f, Loader = Loader)

In [ ]:
fileset = {
            'TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU': samples['TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU']['ntuples'],
            'ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU']['ntuples'],
            'ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU']['ntuples'],
            'ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU']['ntuples'],
            'ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU']['ntuples'],
            'ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU']['ntuples'],
        }

In [ ]:
meta_accumulator = {}
for sample in fileset:
    if sample not in meta_accumulator:
        meta_accumulator.update({sample: processor.defaultdict_accumulator(int)})

In [ ]:
meta_accumulator

In [ ]:
exe = processor.futures_executor
exe_args = {"schema": BaseSchema, "workers": 20, "skipbadfiles": True,}

meta_output_flat_remote = processor.run_uproot_job(
            fileset,
            treename='myana/mytree',
            processor_instance = meta_processor(accumulator=meta_accumulator),
            executor = exe,
            executor_args = exe_args,
            chunksize = 1000000,
            maxchunks = None,
        )

In [ ]:
meta_output_flat_remote['TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU']['sumWeight']=1

In [ ]:
tt_accum = {'TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU'
: processor.defaultdict_accumulator(int)}

In [ ]:
meta = {}
for sample in fileset:
    meta[sample] = meta_output_flat_remote[sample]
    meta[sample]['xsec'] = samples[sample]['xsec']

meta['TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU']['xsec'] = 0

In [ ]:
meta

In [ ]:
from plots.helpers import scale_histos

In [ ]:
from plots.helpers import makePlot2

In [ ]:
import gzip
import pickle
path = '../analysis/test_output.pkl.gz'
with gzip.open(path) as fin:
    output_flat_remote = pickle.load(fin)

In [ ]:
for key in output_flat_remote.keys():
    if type(output_flat_remote[key]) is not type(output_flat_remote['cutflow']):
        output_flat_remote[key] = scale_histos(output_flat_remote[key], meta, fileset, lumi=0.001)

In [ ]:
N_bins = hist.Bin('multiplicity', r'$N$', 6, -0.5, 5.5)
mass_bins = hist.Bin('mass', r'$M\ (GeV)$', 30, 0, 300)
pt_bins = hist.Bin('pt', r'$p_{T}\ (GeV)$', 50, 0, 500)
ht_bins = hist.Bin('pt', r'$H_{T}\ (GeV)$', 50, 0, 2500)
fatjet_pt_bins = hist.Bin('pt', r'$p_{T}\ (GeV)$', 50, 200, 700)
eta_bins = hist.Bin("eta", "$\eta$", 33, -4, 4)
phi_bins = hist.Bin("phi", "$\phi$", 33, -4, 4)
tau_bins = hist.Bin("tau", "$\tau$", 10, 0, 1)

labels ={
    ('ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU',): r'$ZJets\to\nu\nu\ (HT\ 200\ to\ 400)$',
    ('ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU',): r'$ZJets\to\nu\nu\ (HT\ 400\ to\ 600)$',
    ('ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU',): r'$ZJets\to\nu\nu\ (HT\ 600\ to\ 800)$',
    ('ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU',): r'$ZJets\to\nu\nu\ (HT\ 800\ to\ 1200)$',
    ('ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU',): r'$ZJets\to\nu\nu\ (HT\ 1200\ to\ 2500)$',
    ('TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU',): r'$t\bar{t}$',
}

colors ={
    ('ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU',): '#FED23F',
    ('ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU',): '#EB7DB5',
    ('ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU',): '#442288',
    ('ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU',): '#6CA2EA',
    ('ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU',): '#B5D33D',
    ('TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU',): '#355C7D',
}
            
makePlot2(output_flat_remote, 'met', 'pt', pt_bins, r'$MET_{pt}\ (GeV)$', labels, colors, remote=True)
makePlot2(output_flat_remote, 'lead_fatjet_pt', 'pt', fatjet_pt_bins, r'$p_{T}\ (GeV)$', labels, colors, remote=True)
makePlot2(output_flat_remote, 'lead_fatjet_eta', 'eta', eta_bins, r'$\eta$', labels, colors, remote=True)
makePlot2(output_flat_remote, 'lead_fatjet_phi', 'phi', phi_bins, r'$\phi$', labels, colors, remote=True)
makePlot2(output_flat_remote, 'lead_fatjet_mass', 'mass', mass_bins, r'$mass\ (GeV)$', labels, colors, remote=True)
makePlot2(output_flat_remote, 'lead_fatjet_sdmass', 'mass', mass_bins, r'$softdrop\ mass\ (GeV)$', labels, colors, remote=True)
makePlot2(output_flat_remote, 'lead_fatjet_tau1', 'tau', tau_bins, r'$\tau_1$', labels, colors, remote=True)
makePlot2(output_flat_remote, 'lead_fatjet_tau2', 'tau', tau_bins, r'$\tau_2$', labels, colors, remote=True)
makePlot2(output_flat_remote, 'lead_fatjet_tau3', 'tau', tau_bins, r'$\tau_3$', labels, colors, remote=True)
makePlot2(output_flat_remote, 'lead_fatjet_tau4', 'tau', tau_bins, r'$\tau_4$', labels, colors, remote=True)
makePlot2(output_flat_remote, 'nfatjet', 'multiplicity', N_bins, r'$n_{fatjet}$', labels, colors, remote=True)        
makePlot2(output_flat_remote, 'ht', 'pt', ht_bins, r'$H_{T}$', labels, colors, remote=True)

In [ ]:
output_flat_remote['ht'].values()